In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import statistics
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
from scipy.stats import ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

In [ ]:
#fill in your interested variables
category_clinical_varibles = ['sex', 'smoking', 'xxxx']
category_snp_varibles = ['rsxxxx', 'rsxxxx']
continous_clinical_varibles = ['age','bmi', 'xxxx]
criteria_c = 'xxx'  ### according to this, the patients are classified into two groups


In [ ]:
#read file 
df_STATA = pd.read_excel('all_sample_dfclean_transform_forSTATA.xlsx')

In [ ]:
#check the null value 

df_STATA= df_STATA[category_snp_varibles + category_clinical_varibles
                    + continous_clinical_varibles + [criteria_c]
                    + ['sampleID']].dropna(axis=0,how='any')

In [ ]:
#######start to do the statistic analysis#######  
df = df_STATA

In [ ]:
#calculate the p value for snp variables chi-squared
def chi_p_calc(features, dataframe, threshold, cols=['chi', 'p_value']):
    
    result_chi_p = {}

    for feature in features:
        cross_table = pd.crosstab(dataframe[feature],
                                  dataframe[criteria_c], margins = True) 
        #print(cross_table)
        f_obs = []
        for n in np.arange(len(dataframe[feature].value_counts())):
            f_obs.append(cross_table.iloc[n][0:2].values)
            #print(f_obs)
        P = stats.chi2_contingency(f_obs, correction=False)[1]
        #print(stats.chi2_contingency(f_obs, correction=False))
        chi = stats.chi2_contingency(f_obs, correction=False)[0]
        result_chi_p[feature] = [chi, P]
        #print("----------------", result_chi_p)
    df_p = pd.DataFrame.from_dict(result_chi_p, orient = 'index', 
                                  columns = cols)
    df_p = df_p.reset_index().rename(columns = {'index' : 'variables'})
    df_p = df_p.round(decimals = 2)
    
    lessthan = list(df_p[df_p[cols[1]] <= threshold]['variables'])  ##pick up variables if
    ##p value is small enough for future machine learning inputs. 
    ##Here the cut-off value is 0.3
    return df_p, lessthan

figure_snps_genotype, selected_snps_genotype = chi_p_calc(category_snp_varibles, df, 
                                                          0.3, ['chi_genotype', 'p_value_genotype'])


In [ ]:
##switch the df to recessive model##
df_recessive= df[category_snp_varibles]
df_recessive[df_recessive == 1] = 0
df_recessive[df_recessive == 2] = 1
df_recessive = pd.concat([df_recessive, df[criteria_c]], axis = 1)
#new 0 : major + heter  #new 1: minor 


In [ ]:
#calculate the p value for snp variables in recessive model

figure_snps_recessive, selected_snps_recessive = chi_p_calc(category_snp_varibles, df_recessive,
                                                            0.3, ['chi_recessive', 'p_value_recessive'])


In [ ]:
##switch the df to dominat model##
df_dominant= df[category_snp_varibles]
df_dominant[df_dominant == 1] = 'A'
df_dominant[df_dominant == 2] = 'A'
df_dominant[df_dominant == 0] = 'B'
df_dominant[df_dominant == 'A'] = 0
df_dominant[df_dominant == 'B'] = 1
df_dominant = pd.concat([df_dominant, df[criteria_c]], axis =1)
#new 0 : heter + minor  #new 1: major 

In [ ]:
#calculate the p value for snp variables in dominant model
figure_snps_dominant, selected_snps_dominant = chi_p_calc(category_snp_varibles, 
                                                          df_dominant, 0.3, 
                                                         ['chi_dominant', 'p_value_dominant'])

In [ ]:
#calculate the p value for clinical category variables
figure_cy, selected_cy = chi_p_calc(category_clinical_varibles, df, 0.1)

In [ ]:
#calculate the p value for clinical continous variables
result_t_test = {}

responders = df[df[criteria_c]== 1]
non_responders = df[df[criteria_c]== 0]

for feature in continous_clinical_varibles:
    mean_responders = statistics.mean(responders[feature].dropna()) 
    mean_nonresponders = statistics.mean(non_responders[feature].dropna()) 
    SD_responders = statistics.stdev(responders[feature].dropna())   
    SD_nonresponders = statistics.stdev(non_responders[feature].dropna())  
    
    t, p = ttest_ind(responders[feature].dropna(), non_responders[feature].dropna())
    result_t_test[feature] = [mean_responders, SD_responders, mean_nonresponders, SD_nonresponders, t, p]
    
df_p = pd.DataFrame.from_dict(result_t_test, orient = 'index', 
                              columns = ['mean_R', 'SD_R', 'mean_NR', 'SD_NR', 't', 'p_value'])
df_p = df_p.reset_index().rename(columns = {'index' : 'variables'})
df_pnew = df_p.round(decimals = 2)
df_pnew['mean_R ± SD'] =  df_pnew['mean_R'].map(str) + ' ± ' + df_pnew['SD_R'].map(str)
df_pnew['mean_NR ± SD'] =  df_pnew['mean_NR'].map(str) + ' ± ' + df_pnew['SD_NR'].map(str)
df_pnewnew = df_pnew[['variables', 'mean_R ± SD', 'mean_NR ± SD', 't', 'p_value']]

selected_cs = list(df_p[df_p['p_value'] <= 0.1]['variables']) #cc means clinical continous data
figure_cs = df_pnewnew

In [ ]:
#draw the histogram for all the continous variables
plt.figure(figsize=(40, 25))
for i, feature in enumerate(continous_clinical_varibles): 
    plt.subplot(6, 5 ,i +1)
    sns.distplot(df[feature], bins = 20, rug = True)
#plt.savefig('distribution for 6th remission.jpg')

In [ ]:
figure_snps = pd.merge(figure_snps_genotype, figure_snps_recessive, on ='variables')
figure_snps = pd.merge(figure_snps, figure_snps_dominant, on ='variables')
figure_snps_p = figure_snps[['variables', 'p_value_genotype', 'p_value_recessive', 'p_value_dominant']]

In [ ]:
##mark red if p value is below 0.3
def snp_red(val):
    if val<0.3:
        color = 'red'
    else:
        color = 'black'
    return ('color:%s'%color)

figure_snps_p.style.applymap(snp_red, subset = ['p_value_genotype', 'p_value_recessive', 'p_value_dominant']) 

In [ ]:
figure_cy
def clinicalinfo_red(val):
    if val<0.1:
        color = 'red'
    else:
        color = 'black'
    return ('color:%s'%color)

figure_cy.style.applymap(clinicalinfo_red, subset = ['p_value']) 

In [ ]:
figure_cs
figure_cs.style.applymap(clinicalinfo_red, subset = ['p_value']) 

In [ ]:
selected_cs
selected_snps = list(set(selected_snps_dominant + selected_snps_recessive + selected_snps_genotype))
selected_cy

In [ ]:
#########start to preprocessing data for machine learning##########
###drop the samples if the variable is NAN

df = df_STATA_ML[category_snp_varibles + selected_cs + selected_cy + [criteria_c] + ['sampleID']].dropna(axis=0,how='any')
#189
##define a new df

In [ ]:
##here is the dataframe with dummy data
dummy_fields = category_snp_varibles + selected_cy
for each in dummy_fields:
    dummies = pd.get_dummies(df.loc[:, each], prefix = each)
    df = pd.concat([df, dummies], axis = 1)
df.shape

In [ ]:
###dummy list for all snps
dummylist_category_snp_varibles = []
for each in category_snp_varibles:
    dummies = pd.get_dummies(df.loc[:, each], prefix = each)
    dummylist_category_snp_varibles.extend(list(dummies.columns))    
    
###dummy list for selected_cy
dummylist_cy = []
for each in selected_cy:
    dummies = pd.get_dummies(df.loc[:, each], prefix = each)
    dummylist_cy.extend(list(dummies.columns))
    
###dummy list for selected snps
dummylist_selectedsnps= []
for each in selected_snps:
    dummies = pd.get_dummies(df.loc[:, each], prefix = each)
    dummylist_selectedsnps.extend(list(dummies.columns))  

In [ ]:
##########start machine learning##########

#NO.1-include clinical data and 24 snps features decision tree
def prediction_DT_calc(inputs, iteration=100):
    resultlist = [] 
    for i in range(iteration):
        X = df[inputs]
        y = df[criteria_c]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
        dtree = DecisionTreeClassifier()
        fit = dtree.fit(X_train,y_train)
        result = fit.score(X_test, y_test)
        resultlist.append(result)
        
    mean = statistics.mean(resultlist) * 100
    SD = statistics.stdev(resultlist) *100
    SE = SD/np.sqrt(len(y_test))
    return {'mean': mean, "SD": SD, "SE": SE}
   
stat1 = prediction_DT_calc(dummylist_category_snp_varibles
                                                 + dummylist_cy + selected_cs)

stat1['index'] = "combo1"
stat1

In [ ]:
#No.1-include clnical data and all 24 SNPs features Random forest

def prediction_RF_calc(inputs, iteration=100):
    resultlist_RF = []
    for i in range(iteration):
        X = df[inputs]
        y = df[criteria_c]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
        rfc = RandomForestClassifier(n_estimators=100)
        rfc.fit(X_train, y_train)
        rfc_pred = rfc.predict(X_test)
        maybeIneed = confusion_matrix(y_test,rfc_pred)
        report = classification_report(y_test,rfc_pred)
        
        result_RF = pd.DataFrame(
            classification_report(y_test,rfc_pred, output_dict=True)
        ).transpose().loc['accuracy', 'f1-score']
        
        resultlist_RF.append(result_RF)

    mean = statistics.mean(resultlist_RF) * 100
    SD = statistics.stdev(resultlist_RF) *100
    SE = SD/np.sqrt(len(y_test))
    return {'mean': mean, "SD": SD, "SE": SE}, (report, X_train, y_train, X_test, y_test)

stat_rf_1, info = prediction_RF_calc(
    dummylist_category_snp_varibles + dummylist_cy + selected_cs)

stat_rf_1['index'] = "combo1"
stat_rf_1


In [ ]:

#No.1-include all snp features--ROC
def drawROC(info):
    rfc = RandomForestClassifier(n_estimators=100)
    fit = rfc.fit(info[1], info[2])
    # ROC
    y_score = rfc.fit(info[1], info[2]).predict_proba(info[3])  
    fpr, tpr, thresholds = roc_curve(info[4], y_score[:, 1])
    roc = auc(fpr, tpr)
    plt.subplots(figsize=(7, 5.5))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

drawROC(info)

In [ ]:
#No.2-include selected snp features_decsion tree
stat2 = prediction_DT_calc(dummylist_selectedsnps + dummylist_cy + selected_cs)
stat2['index'] = "combo2"
stat2

In [ ]:
#No.2-include selected snp features_random forest
stat_rf_2, info = prediction_RF_calc(
    dummylist_selectedsnps + dummylist_cy + selected_cs)
stat_rf_2['index'] = "combo2"
stat_rf_2

In [ ]:
#NO3.only include non-snp data_decision tree
stat3 = prediction_DT_calc(dummylist_cy + selected_cs)
stat3['index'] = "combo3"
stat3

In [ ]:
#No.3 only include non-snp_random forest
stat_rf_3, info = prediction_RF_calc(
    dummylist_cy + selected_cs)
stat_rf_3['index'] = "combo3"
stat_rf_3

In [ ]:
#NO.4 only include selected snp features_decision tree
stat4 = prediction_DT_calc(dummylist_selectedsnps)
stat4['index'] = "combo4"
stat4

In [ ]:
#NO.4-only include selected snp features_random forest
stat_rf_4, info = prediction_RF_calc(
    dummylist_selectedsnps)
stat_rf_4['index'] = "combo4"
stat_rf_4

In [ ]:
#NO.5 include all 24 the snps features decision tree
stat5 = prediction_DT_calc(dummylist_category_snp_varibles)
stat5['index'] = "combo5"
stat5

In [ ]:
#No.5 only include all 24 SNPs features Random forest

stat_rf_5, info = prediction_RF_calc(
    dummylist_category_snp_varibles)
stat_rf_5['index'] = "combo5"
stat_rf_5

In [ ]:
final_dt = pd.DataFrame([stat1])
df_dt = final_dt.append([stat2]).append([stat3]).append([stat4]).append([stat5])
df_dt.index = df_dt['index']
df_dt['details'] = ['selected clinical score + ALL snps', 'selected clinical score+ selected snps',
                    'selected clinical score', 'selected snps', 'ALL snps']
df_dt_final = df_dt[['details', 'mean', 'SD', 'SE']]
df_dt_final

In [ ]:
final_rf = pd.DataFrame([stat_rf_1])
df_rf = final_rf.append([stat_rf_2]).append([stat_rf_3]).append([stat_rf_4]).append([stat_rf_5])
df_rf.index = df_rf['index']
df_rf['details'] = ['selected clinical score + ALL snps', 'selected clinical score+ selected snps',
                    'selected clinical score', 'selected snps', 'ALL snps']
df_rf_final = df_rf[['details', 'mean', 'SD', 'SE']]
df_rf_final


In [ ]:
sns.barplot( x = df_rf_final.index, y = 'mean', data = df_rf_final) 